# GLO-MIA adapted to use logits instead of label only
Relaxes the constraint that we only get labels, providing logits or probabilities to the model. (Is getting probabilities back a realistic situation?)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

from torch.nn import functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset
from torch_geometric.loader import DataLoader as GDataLoader
from torch_geometric.datasets import TUDataset
from sklearn.utils.class_weight import compute_class_weight

from ml_util import GATProteinsModel, train_model_multi_graph, train_model, load_model
from util import onehot_transform, graph_train_test_split, create_perturbed_graphs

In [3]:
DEVICE = ('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
def _get_onehot_transform(n_categories):
    return lambda x: onehot_transform(x, categories=list(range(n_categories)))

In [8]:
n_categories = 2
num_feat = 3

dataset = TUDataset(root='/home/hice1/khom9/scratch/CSE-8803-MLG-Data/PROTEINS', name='PROTEINS', transform=_get_onehot_transform(n_categories))
# dataset = TUDataset(root='/home/hice1/khom9/scratch/CSE-8803-MLG-Data/DD', name='DD', transform=_get_onehot_transform(n_categories))
dataset = onehot_transform(dataset)

In [9]:
# Split dataset in half for the target model dataset and shadow model dataset
t_dataset, s_dataset = graph_train_test_split(dataset, test_size=0.5)

# Split each dataset into train/test splits
t_dataset_train, t_dataset_test = graph_train_test_split(t_dataset, test_size=0.5)
s_dataset_train, s_dataset_test = graph_train_test_split(s_dataset, test_size=0.5)

In [11]:
# Create and train target model
lr = 0.001
epochs = 100
batch_size = 8
weight_decay = 1e-4
t_model = GATProteinsModel(num_feat=num_feat, num_classes=n_categories).to(DEVICE)
optimizer = optim.Adam(t_model.parameters(), lr=lr, weight_decay=weight_decay)
t_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.5,
                                                       patience=50,
                                                       min_lr=1e-6,
                                                       verbose=True)
weight = compute_class_weight('balanced', classes=np.unique(t_dataset_train.y.argmax(dim=1)), y=t_dataset_train.y.argmax(dim=1).numpy())
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(weight).to(DEVICE))
# t_save_path = 'mia-models/t_model_proteins.pth'
t_save_path = None

train_model_multi_graph(t_model, optimizer, t_dataset_train, loss_fn, epochs, batch_size, val_dataset=t_dataset_test, save_path=t_save_path, save_freq=10, scheduler=t_scheduler, device=DEVICE)
# t_model = load_model(t_model, t_save_path)

Learning rate: 0.001
Scheduler: <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x15520dd662c0>
Training for 100 epochs, with batch size=8
Using validation data (278 samples)
Using device: cuda:0

-----Epoch 1/100-----
Batch 10/35 | loss: 0.70198 (0.119s) | train acc: 0.500, train AUC: 0.532
Batch 20/35 | loss: 0.68083 (0.117s) | train acc: 0.544, train AUC: 0.556
Batch 30/35 | loss: 0.71027 (0.117s) | train acc: 0.562, train AUC: 0.558
Batch 35/35 | loss: 0.65569 (0.057s) | train acc: 0.568, train AUC: 0.571
Validation: val loss: 0.642 | val acc: 0.658 | val F1: 0.492 | val AUC: 0.718

-----Epoch 2/100-----
Batch 10/35 | loss: 0.66536 (0.116s) | train acc: 0.650, train AUC: 0.619
Batch 20/35 | loss: 0.66300 (0.117s) | train acc: 0.656, train AUC: 0.669
Batch 30/35 | loss: 0.66242 (0.116s) | train acc: 0.642, train AUC: 0.653
Batch 35/35 | loss: 0.66182 (0.057s) | train acc: 0.633, train AUC: 0.647
Validation: val loss: 0.585 | val acc: 0.723 | val F1: 0.698 | val AUC: 0.773

---

In [12]:
# Create and train shadow model
lr = 0.001
epochs = 100
batch_size = 8
weight_decay = 1e-4
s_model = GATProteinsModel(num_feat=num_feat, num_classes=n_categories).to(DEVICE)
optimizer = optim.Adam(s_model.parameters(), lr=lr, weight_decay=weight_decay)
s_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.5,
                                                       patience=50,
                                                       min_lr=1e-6,
                                                       verbose=True)
weight = compute_class_weight('balanced', classes=np.unique(s_dataset_train.y.argmax(dim=1)), y=s_dataset_train.y.argmax(dim=1).numpy())
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(weight).to(DEVICE))
# s_save_path = 'mia-models/s_model_proteins.pth'
s_save_path = None

train_model_multi_graph(s_model, optimizer, s_dataset_train, loss_fn, epochs, batch_size, val_dataset=s_dataset_test, save_path=s_save_path, save_freq=10, device=DEVICE)
# s_model = load_model(s_model, s_save_path)

Learning rate: 0.001
No learning rate scheduling!
Training for 100 epochs, with batch size=8
Using validation data (279 samples)
Using device: cuda:0

-----Epoch 1/100-----
Batch 10/35 | loss: 0.66077 (0.120s) | train acc: 0.637, train AUC: 0.651
Batch 20/35 | loss: 0.71905 (0.119s) | train acc: 0.588, train AUC: 0.601
Batch 30/35 | loss: 0.68674 (0.119s) | train acc: 0.592, train AUC: 0.600
Batch 35/35 | loss: 0.63976 (0.058s) | train acc: 0.594, train AUC: 0.605
Validation: val loss: 0.638 | val acc: 0.677 | val F1: 0.567 | val AUC: 0.702

-----Epoch 2/100-----
Batch 10/35 | loss: 0.60612 (0.112s) | train acc: 0.700, train AUC: 0.726
Batch 20/35 | loss: 0.64632 (0.113s) | train acc: 0.706, train AUC: 0.723
Batch 30/35 | loss: 0.61107 (0.113s) | train acc: 0.696, train AUC: 0.725
Batch 35/35 | loss: 0.55963 (0.055s) | train acc: 0.705, train AUC: 0.732
Validation: val loss: 0.677 | val acc: 0.577 | val F1: 0.599 | val AUC: 0.657

-----Epoch 3/100-----
Batch 10/35 | loss: 0.55303 (0.11

In [13]:
def create_attack_dataset(model, train_dataset, test_dataset, scaler=0.4, n_perturb_per_graph=10, device='cpu'):
    '''Create an attack dataset containing logits that are generated from perturbed graphs'''
    
    model.eval()
    chunksize = 250
    train_loader = GDataLoader(train_dataset, batch_size=chunksize, shuffle=False)
    test_loader = GDataLoader(test_dataset, batch_size=chunksize, shuffle=False)
    all_logits = []
    all_labels = []
    
    with torch.no_grad():
        # Enumerate over each dataset
        for i, loader in enumerate((train_loader, test_loader)):
            label = [1,0] if i == 0 else [0,1]
            for i, gbatch in enumerate(loader):
                gbatch = gbatch.to(device)
                x_p = create_perturbed_graphs(gbatch.x, num=n_perturb_per_graph, scaler=scaler, device=device).to(device)
                pred = torch.stack([model(x_pi, gbatch.edge_index, gbatch.batch).squeeze() for x_pi in x_p])
                all_logits.append((pred.flatten(end_dim=1).sort(dim=1)[0]))
                all_labels.append(torch.Tensor([label] * n_perturb_per_graph * len(gbatch)))        
        
            
    return torch.cat(all_logits), torch.cat(all_labels)

In [19]:
class GenericAttackModel(nn.Module):
    def __init__(self, num_feat):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_feat, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
        
        
    def forward(self, x):
        return self.layers(x)

In [20]:
s_logits, s_labels = create_attack_dataset(s_model, s_dataset_train, s_dataset_test, scaler=.2, device=DEVICE)
t_logits, t_labels = create_attack_dataset(t_model, t_dataset_train, t_dataset_test, scaler=.2, device=DEVICE)

In [21]:
att_dataset = TensorDataset(s_logits, s_labels)
target_dataset = TensorDataset(t_logits, t_labels)
lr = 0.005
epochs = 150
batch_size = 32
att_model = GenericAttackModel(num_feat=2).to(DEVICE)
optimizer = optim.Adam(att_model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

train_model(att_model, optimizer, att_dataset, loss_fn, epochs, batch_size, val_dataset=target_dataset, device=DEVICE)
att_model.eval()

Learning rate: 0.005
No learning rate scheduling!
Training for 150 epochs, with batch size=32
Using device: cuda:0

-----Epoch 1/150-----
Batch 50/175, loss: 0.6966510212421417 (0.055s), train acc: 0.496
 | val loss: 0.693 | val acc: 0.539 | val F1: 0.444 | val AUC: 0.539
Batch 100/175, loss: 0.693475935459137 (0.054s), train acc: 0.494
 | val loss: 0.693 | val acc: 0.504 | val F1: 0.638 | val AUC: 0.541
Batch 150/175, loss: 0.6924813306331634 (0.055s), train acc: 0.508
 | val loss: 0.692 | val acc: 0.523 | val F1: 0.526 | val AUC: 0.538
Batch 175/175, loss: 0.6923344731330872 (0.028s), train acc: 0.509
 | val loss: 0.694 | val acc: 0.500 | val F1: 0.667 | val AUC: 0.500

-----Epoch 2/150-----
Batch 50/175, loss: 0.6938857114315033 (0.054s), train acc: 0.508
 | val loss: 0.694 | val acc: 0.500 | val F1: 0.667 | val AUC: 0.500
Batch 100/175, loss: 0.6931924426555633 (0.054s), train acc: 0.508
 | val loss: 0.693 | val acc: 0.500 | val F1: 0.667 | val AUC: 0.500
Batch 150/175, loss: 0.693

KeyboardInterrupt: 

In [ ]:
ddd = TensorDataset(logits, labels)

In [ ]:
ddd[0:2]